## Limpieza de archivo de despacho para liquidación

In [2]:
# Llamado de librerias
import pandas as pd

df_dsp = pd.read_excel('dspcttos/dspcttos_tx2.xlsx') 
df_cliq = pd.read_excel('cliq/cliq_tx2.xlsx')
df_dsp = df_dsp.round(2)


In [3]:
# Eliminación de columnas que no son necesarrias para el despacho
df = df_dsp.drop(['TIPO','TIPOMERC','TIPO ASIGNA'],axis=1)

In [4]:
# Se crea lista de columnas pra seleccionar
_desp = ['DESP_HORA {:02}'.format(i) for i in range(1,25)]
_hora = ['TRF_HORA {:02}'.format(i) for i in range(1,25)]

# Contratos de compra y venta
Compras_Contratos = df[df['COMPRADOR'].isin(['SFEC'])]
Ventas_Contratos = df[df['VENDEDOR'].isin(['SFEC'])]

# Filtro para filtrar por el agente comercializador
cliq= df_cliq[df_cliq['AGENTE'].isin(['SFEC'])]
# Bolsa
ventas_bolsa_kWh = cliq['VENTAS BOLSA kwh'].sum()
ventas_bolsa_COP = cliq['VENTAS BOLSA $'].sum()
compras_bolsa_kWh = cliq['COMPRAS BOLSA kwh'].sum()
compras_bolsa_COP = -cliq['COMPRAS BOLSA $'].sum()

In [5]:
# Mostrar consolidado en bolsa
print(' ventas en bolsa kWh/mes: {:,.2f}'.format(ventas_bolsa_kWh))
print(f' ventas en bolsa COP/mes: ${ventas_bolsa_COP:,.2f}')
print(' compras en bolsa kWh/mes: {:,.2f}'.format(compras_bolsa_kWh))
print(f' compras en bolsa kWh/mes: ${compras_bolsa_COP:,.2f}')

 ventas en bolsa kWh/mes: 1,446,945.83
 ventas en bolsa COP/mes: $1,314,519,278.51
 compras en bolsa kWh/mes: 292.92
 compras en bolsa kWh/mes: $-273,773.93


In [6]:
#Se divide en contratos de compra y bolsa
Compras_Contratos_kWh = Compras_Contratos.groupby(['CONTRATO','COMPRADOR','VENDEDOR'])[_desp].sum().sum(axis=1).reset_index(name='Total [kWh/mes]')
Ventas_Contratos_kWh = Ventas_Contratos.groupby(['CONTRATO','COMPRADOR','VENDEDOR'])[_desp].sum().sum(axis=1).reset_index(name='Total [kWh/mes]')

In [7]:
# Se calcula el despacho de los contratos
Compras_Contratos_COP = Compras_Contratos.groupby(['CONTRATO','COMPRADOR','VENDEDOR']).apply(lambda x:(-x[_desp].values*x[_hora].values).sum()).reset_index(name='Total [COP]')
Ventas_Contratos_COP = Ventas_Contratos.groupby(['CONTRATO','COMPRADOR','VENDEDOR']).apply(lambda x:(x[_desp].values*x[_hora].values).sum()).reset_index(name='Total [COP]')

/tmp/ipykernel_190638/3208950553.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  Compras_Contratos_COP = Compras_Contratos.groupby(['CONTRATO','COMPRADOR','VENDEDOR']).apply(lambda x:(-x[_desp].values*x[_hora].values).sum()).reset_index(name='Total [COP]')
/tmp/ipykernel_190638/3208950553.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  Ventas_Contratos_COP = Ventas_Contratos.groupby(['CONTRATO','COM

In [8]:
# Paso anterior no es necesario para el calculo pero si se quiere mostar tablas diferentes (compras y ventas) es un buen ejercicio
Compras = pd.merge(Compras_Contratos_kWh,Compras_Contratos_COP,on=['CONTRATO','COMPRADOR','VENDEDOR'],how='inner')
Ventas = pd.merge(Ventas_Contratos_kWh,Ventas_Contratos_COP,on=['CONTRATO','COMPRADOR','VENDEDOR'],how='inner')

Compras_COP=Compras['Total [COP]'].sum()
ventas_COP=Ventas['Total [COP]'].sum()

Compras_kwh=Compras['Total [kWh/mes]'].sum()
ventas_kwh=Ventas['Total [kWh/mes]'].sum()
# Se realiza un consolidado completo de todos los contratos del comercializador
totales = pd.DataFrame([['COMPRAS CONTRATOS','SFEC', '-',Compras_kwh,Compras_COP],['VENTAS CONTRATOS','-', 'SFEC',ventas_kwh,ventas_COP],
                        ['COMPRAS BOLSA','SFEC','-',compras_bolsa_kWh,compras_bolsa_COP],['VENTAS BOLSA','-', 'SFEC',ventas_bolsa_kWh,ventas_bolsa_COP],
                        ['CONSOLIDADO','-','-',Compras_kwh+compras_bolsa_kWh,ventas_COP+ventas_bolsa_COP+Compras_COP+compras_bolsa_COP]], columns=Ventas.columns)

resultado_contratos = pd.merge(Compras,Ventas,on=['CONTRATO','COMPRADOR','VENDEDOR','Total [kWh/mes]','Total [COP]'],how='outer')
resultado_contratos = pd.concat([resultado_contratos,totales])
resultado_contratos['Total [kWh/mes]'] = resultado_contratos['Total [kWh/mes]'].apply(lambda x: f'{x:,.2f}')
resultado_contratos['Total [COP]'] = resultado_contratos['Total [COP]'].apply(lambda x: f'${x:,.2f}')

In [9]:
import matplotlib.pyplot as plt

def crear_tabla_imagen(dataframe, nombre_archivo):
    fig, ax = plt.subplots(figsize=(10, 2))
    ax.axis('tight')
    ax.axis('off')
    
    # Crear la tabla
    tabla = ax.table(cellText=dataframe.values, colLabels=dataframe.columns, cellLoc='center', loc='center')
    tabla.auto_set_font_size(False)
    tabla.set_fontsize(9)

    # Establecer el formato de los títulos
    for (i, j), cell in tabla.get_celld().items():
        if i == 0 or i == len(dataframe):  # Títulos
            cell.set_text_props(fontweight='bold', color='black')
        else:
            cell.set_text_props(fontweight='normal', color='black')

    # Guardar la tabla como imagen
    plt.savefig(nombre_archivo, bbox_inches='tight', dpi=500)
    plt.close()

# Crear imagen de la tabla
crear_tabla_imagen(resultado_contratos, 'tabla_liquidacion.png')

In [10]:
fin_mes =((ventas_COP+ventas_bolsa_COP+Compras_COP+compras_bolsa_COP)/len(cliq))*30
# Total del fin de mes en COP de balance o consolidado del comercializador
f'{fin_mes:,.2f}'


'1,809,936,005.69'